<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Financial Data Science &mdash; Viz Streaming

**DataNatives Berlin 2015**

Dr Yves J Hilpisch

<a href='mailto:team@tpq.io'>team@tpq.io</a> | <a href='http://tpq.io'>http://tpq.io</a>

The Python Quants GmbH

## Prerequisites

In [ ]:
import json
import time
import pandas as pd
import requests
import plotly.plotly as py
import plotly.tools as tls 
from plotly.graph_objs import *
import cufflinks

Loading **Plotly credentials** from file.

In [ ]:
pcreds = json.load(open('plotly_creds'))

In [ ]:
py.sign_in('yves', pcreds['api_key'])

## Dummy Tick Data Source

For illustration purposes, we use this data source which provides **dummy tick data**.

In [ ]:
tick_url = 'http://cloud.datapark.io:12500/prices'

In [ ]:
r = requests.get(tick_url)
r.json()

## Collecting Data from Source

Collecting **data via a loop**.

In [ ]:
df = pd.DataFrame()
for _ in xrange(100):
    r = requests.get(tick_url)
    data = r.json()
    df = df.append(pd.DataFrame(data['tick'],
                                index=(data['tick']['time'],)))

In [ ]:
df.tail()

## Plotting with plotly

Converting the `DataFrame` data into **Plotly compatible format**.

In [ ]:
def df_to_iplot(df):
    '''
    Coverting a pandas DataFrame to Plotly interface.
    '''
    x = df.index.values
    lines = {}
    for key in df:
        lines[key] = {}
        lines[key]["x"] = x
        lines[key]["y"] = df[key].values
        lines[key]["name"] = key
        # Appending all lines
    lines_plotly=[lines[key] for key in df]
    return lines_plotly

And **iplotting** it.

In [ ]:
py.iplot(df_to_iplot(df[['ask']]))

## More Simple with Cufflinks

Cufflinks wraps the Plotly API for the `DataFrame` class.

In [ ]:
df['ask'].iplot(world_readable=True)

## Streaming Plot

Plotting **data streams** is a bit more involved. We need a `Stream` object ...

In [ ]:
# Get stream id from stream id list 
stream_ids = pcreds['stream_ids']

# Make instance of stream id object 
stream_0 = Stream(
    token=stream_ids[0],
    maxpoints=150)

... and a `Scatter` object which gets passed to a `Data` object.

In [ ]:
trace0 = Scatter(
    x=[], y=[],
    mode='lines+markers',
    stream=stream_0,
    name='price')

dats = Data([trace0])

The rest then is again straightforward.

In [ ]:
layout = Layout(title='Streaming Plot')
fig = Figure(data=dats, layout=layout)
unique_url = py.plot(fig, filename='stream_plot', auto_open=False)

**Embedding** the plot first ...

In [ ]:
tls.embed(unique_url)

... **streaming** the data second.

In [ ]:
s0 = py.Stream(stream_ids[0])
s0.open()
for i in xrange(250):
    r = requests.get(tick_url)
    data = r.json()
    s0.write({'x': data['tick']['time'][11:-3], 'y': data['tick']['ask']})
    time.sleep(0.25)

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@pqp.io">team@tpq.io</a>

**Quant Platform** |
<a href="http://quant-platform.com">http://quant-platform.com</a>

**datapark.io** |
<a href="http://datapark.io">http://datapark.io</a>

**Python for Finance** |
<a href="http://python-for-finance.com" target="_blank">Python for Finance @ O'Reilly</a>

**Derivatives Analytics with Python** |
<a href="http://derivatives-analytics-with-python.com" target="_blank">Derivatives Analytics @ Wiley Finance</a>